###**Importing necessary Libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

###**Device Configuration**

In [ ]:
# check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
# Load the breast cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

In [ ]:
print(X)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [ ]:
print(y[:20])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [ ]:
# Splitting the dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(569, 30)
(483, 30)
(86, 30)


In [ ]:
# Standardizing the data using Standard scaler to avoid data leakage and normally distributed
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
# Convert data to PyTorch tensors and move it to GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

###**Building Neural Network Architecture**

In [ ]:
# Define the neural network architecture
class NeuralNet(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNet, self).__init__()
    self.fc1  = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

In [ ]:
print(X_train.shape[1])

30


In [ ]:
# Define the hyperparameters
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100

In [ ]:
# Initiliaze the neural network and move it to the GPU
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [ ]:
# Define the loss and the optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

###**Training the Neural Network**

In [ ]:
# Training the model
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criterion(outputs, y_train.view(-1, 1))
  loss.backward()
  optimizer.step()

  # Evaluate accuracy
  with torch.no_grad():
    model.eval()
    outputs = model(X_test)
    predicted = (outputs.round() > 0.5).float()
    acc = (predicted == y_test.view(-1, 1)).float().mean()
    correct = (predicted == y_test.view(-1, 1)).float().sum()
    accuracy = correct / y_test.size(0)
    total = y_test.size(0)

  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item()*100:.2f}%")

Epoch [10/100], Loss: 0.5584, Accuracy: 87.21%
Epoch [20/100], Loss: 0.4303, Accuracy: 95.35%
Epoch [30/100], Loss: 0.3373, Accuracy: 95.35%
Epoch [40/100], Loss: 0.2688, Accuracy: 96.51%
Epoch [50/100], Loss: 0.2193, Accuracy: 96.51%
Epoch [60/100], Loss: 0.1841, Accuracy: 96.51%
Epoch [70/100], Loss: 0.1587, Accuracy: 96.51%
Epoch [80/100], Loss: 0.1398, Accuracy: 96.51%
Epoch [90/100], Loss: 0.1254, Accuracy: 98.84%
Epoch [100/100], Loss: 0.1140, Accuracy: 98.84%


###**Model Evaluation**

In [ ]:
# Evaluation on Training set
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  accuracy = (predicted == y_train.view(-1, 1)).float().sum()
  accuracy = accuracy / y_train.size(0)
  print(f"Training Accuracy: {accuracy.item()*100:.2f}%")

Training Accuracy: 97.72%


In [ ]:
# Evaluation on Training set
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  accuracy = (predicted == y_train.view(-1, 1)).float().mean()
  print(f"Training Accuracy: {accuracy.item()*100:.2f}%")

Training Accuracy: 97.72%


In [ ]:
# Evaluation on Test set
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  accuracy = (predicted == y_test.view(-1, 1)).float().sum()
  accuracy = accuracy / y_test.size(0)
  print(f"Test Accuracy: {accuracy.item()*100:.2f}%")

Test Accuracy: 98.84%


In [ ]:
# Evaluation on Test set
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  accuracy = (predicted == y_test.view(-1, 1)).float().mean()
  print(f"Test Accuracy: {accuracy.item()*100:.2f}%")

Test Accuracy: 98.84%
